In [60]:
#!pip install pandas tqdm

import pandas as pd
from tqdm import tqdm
import re


from google.colab import drive
drive.mount('/content/drive')

In [100]:
# 파일 경로 지정 (본인 경로에 맞게 수정)
file_path = 'Ontology_filtered_reviews.txt'

# 드라마명과 리뷰 추출
drama_names = []
reviews = []

with open(file_path, encoding='utf-8') as f:
    current_drama = None
    for line in f:
        line = line.strip()
        if not line:
            continue
        if line.startswith('[') and line.endswith(']'):
            current_drama = line[1:-1]  # 대괄호 제거
        else:
            if current_drama is not None and line:
                drama_names.append(current_drama)
                reviews.append(line)

# 데이터프레임 생성
df = pd.DataFrame({'drama': drama_names, 'review': reviews})
print(f"총 {len(df)}개의 리뷰를 불러왔습니다.")
print(df.head())

총 2135개의 리뷰를 불러왔습니다.
                 drama                                             review
0  (아는 건 별로 없지만) 가족입니다  흔하디 흔한 자극적 전개가 아닐까싶었지만 볼수록 오묘하고 생각하게 만드는 이야기들이...
1  (아는 건 별로 없지만) 가족입니다                                   가족은 같이 살아온 세월이다.
2              1%의 어떤것                                                 음.
3               18 어게인  뭐 유치하지만 재밌다 가볍게 보기좋다 드라마가 재밌음댔지 갑자기 교훈 +감동+ 억지...
4               18 어게인                                     눈물이 났다 왠지 모르게…


In [106]:
df['review']

0       흔하디 흔한 자극적 전개가 아닐까싶었지만 볼수록 오묘하고 생각하게 만드는 이야기들이...
1                                        가족은 같이 살아온 세월이다.
2                                                      음.
3       뭐 유치하지만 재밌다 가볍게 보기좋다 드라마가 재밌음댔지 갑자기 교훈 +감동+ 억지...
4                                          눈물이 났다 왠지 모르게…
                              ...                        
2130                            깊진 않지만 은은한 커피향처럼 산뜻한 드라마.
2131                                        비로소 '나'와 화해하다
2132                                              몰입이 안됨.
2133                   살면서 내가 아주 많이 힘든 날이 온다면 다시 꺼내볼 것 같다
2134    i enjoyed it so much.. maybe because im turnin...
Name: review, Length: 2135, dtype: object

In [108]:
import re

def clean_text(text):
    # 한글, 영어, 숫자, 공백만 남기고 모두 제거
    return re.sub(r'[^\w\s가-힣]', '', str(text))


In [112]:
# 명확한 부정 의미의 키워드 (필요시 추가/수정)
negative_keywords = [
    '재미없', '노잼', '불편', '불만', '짜증', '실망', '별로', '최악', '지루', '유치해', 
    '답답', '이상', '비추', '불쾌', '싫', '아쉽', '꺼려', '나오고싶', '중간에 보다가', '그냥 그렇', 
    '아니', '할말없', '못', '아닌', '망작', '실패', '미흡', '부족', '어이없', '흥미가 계속되지않는다', 
    '집중은 힘들 것 같다', '안됨', '으악','ㅡㅡ','안본다','왜봤지','이상해','심하다','공감되는 건','유머요',
    '씁쓸함','음썽','집중이','집중을','아까워','안됨','의리로','길었어','집중이 잘 안된다','잃는다','어수선하다','하차',
    '아깝','우롱','장난하나','꺼져','잼있어?','우울','산으로','아쉬움','모르겠네요','왜봤지','개판이네','재미가 없다','조잡하다',
    '몰입이 안됨','장난해?','이해안돼','초반에만','스트레스받아','정신차려라','시간 아깝다','쓸데없이','정신없어요','뭐에요;','가볍다.','시작은']



In [109]:
def is_negative(review):
    review_clean = clean_text(review)
    for kw in negative_keywords:
        if clean_text(kw) in review_clean:
            return True
    return False


In [113]:
for reviews in df['review']:
    print(f"{reviews} -> {is_negative(reviews)}")


흔하디 흔한 자극적 전개가 아닐까싶었지만 볼수록 오묘하고 생각하게 만드는 이야기들이 흥미롭다 -> False
가족은 같이 살아온 세월이다. -> False
음. -> False
뭐 유치하지만 재밌다 가볍게 보기좋다 드라마가 재밌음댔지 갑자기 교훈 +감동+ 억지 눈물만 아님 좋겠다 -> False
눈물이 났다 왠지 모르게… -> False
기분 좋아지는 휴먼 판타지 이도현의 발견, 찐 농구 장면 인정~ -> False
잔잔할수록 깊다. -> False
두근두근 설렌다 -> False
마음이 편안하고 따뜻해지는 그것만으로도 충분했던 -> False
지나온 어제보다 오늘을 믿고, 알 수 없는 내일을 향해 용기를 내는 것. 주어진 오늘에 최선을 다해야 할 이유가 여기에 있다. -> False
참신 신선 군더더기없는 스토리... -> False
말로 다하지 않아도 느껴지는 진심.  서서히 스며와 나마저도 알지 못한 시간 동안 지그시 바라봐주던 따스한 마음. -> True
끝까지 힘을 잃지 않는 유머의 동력. -> False
억지 감동과 쓸데없는 진지함을 내려놓고 '성실하게' 웃긴다. -> False
으악 유치해 -> True
간만에 정말 아무 생각 안하고 즐겁게 웃으며 시간 보낼 수 있었다. 자기 본연과 해야할 일에 충실했다. -> False
제법 많이 웃겼음. 칭찬해. -> False
생각없이 유쾌하게 보기엔 좋다. -> False
속보이는 코미디의 경쾌한 잰걸음 -> False
억지 감동이 아닌 웃음으로만 밀어붙인 것이 인상적. -> True
극한직업 이전 간만에 본 수준급 코미디. -> False
전형적인 한국 코미디 작품. -> False
꺼려질 만큼 싸보이지도 부담스러울 정도로 비싸보이지도 않았던 경쾌한 영화. -> True
두 사람 케미가 마음에 들었었다.. -> False
당시엔 꽤 웃었던 것 같은데 다시 보곤 두 번 정도 웃었다. 뜬금없는 로케이션 변환이 특색이다. -> False
중간에 보다가 나오고싶었음 ㅡㅡ -> True
유머요...? -> True

In [114]:
# 부정 리뷰 제거
# 긍정 리뷰!!
filtered_df = df[~df['review'].apply(is_negative)].reset_index(drop=True)

print(f"최종 긍정/힐링 리뷰 개수: {len(filtered_df)}")
print(filtered_df)


최종 긍정/힐링 리뷰 개수: 1934
                    drama                                             review
0     (아는 건 별로 없지만) 가족입니다  흔하디 흔한 자극적 전개가 아닐까싶었지만 볼수록 오묘하고 생각하게 만드는 이야기들이...
1     (아는 건 별로 없지만) 가족입니다                                   가족은 같이 살아온 세월이다.
2                 1%의 어떤것                                                 음.
3                  18 어게인  뭐 유치하지만 재밌다 가볍게 보기좋다 드라마가 재밌음댔지 갑자기 교훈 +감동+ 억지...
4                  18 어게인                                     눈물이 났다 왠지 모르게…
...                   ...                                                ...
1929           커피 한잔 할까요?                       잔잔하게, 의외로 뻔하게 흘러가지 않아서 괜찮았던.
1930           커피 한잔 할까요?                          깊진 않지만 은은한 커피향처럼 산뜻한 드라마.
1931              안녕? 나야!                                      비로소 '나'와 화해하다
1932              안녕? 나야!                 살면서 내가 아주 많이 힘든 날이 온다면 다시 꺼내볼 것 같다
1933              안녕? 나야!  i enjoyed it so much.. maybe because im turnin...

[1934 rows x 2 columns]


In [116]:
filtered_df.to_csv('Ontology_positive_reviews4.csv', index=False, encoding='utf-8-sig')


In [117]:
filtered_df.drama.value_counts()

drama
우리는 오늘부터    19
7급 공무원      14
흑기사         14
번외수사        14
아이템         13
            ..
블랙           1
불가살          1
부잣집 아들       1
부부의 세계       1
스포트라이트       1
Name: count, Length: 602, dtype: int64

In [118]:
negative_df = df[df['review'].apply(is_negative)].reset_index(drop=True)
print(f"추출된 부정 리뷰 개수: {len(negative_df)}")
print(negative_df)

추출된 부정 리뷰 개수: 201
              drama                                             review
0               49일  말로 다하지 않아도 느껴지는 진심.  서서히 스며와 나마저도 알지 못한 시간 동안 ...
1            7급 공무원                                             으악 유치해
2            7급 공무원                       억지 감동이 아닌 웃음으로만 밀어붙인 것이 인상적.
3            7급 공무원          꺼려질 만큼 싸보이지도 부담스러울 정도로 비싸보이지도 않았던 경쾌한 영화.
4            7급 공무원                                  중간에 보다가 나오고싶었음 ㅡㅡ
..              ...                                                ...
196              화랑                         재미없다 재미없다 하다가 끝을 봤네... 만세!
197  회사를 관두는 최고의 순간                                      에휴 ㅈㄴ 우울띠하다 걍
198             흑기사                                     코미디인데 왜 웃기질 못해
199              XX                                    생각 해보지 못했던 이야기.
200         안녕? 나야!                                            몰입이 안됨.

[201 rows x 2 columns]


In [119]:
negative_df.drama.value_counts()

drama
7급 공무원            5
날 녹여주오            3
날씨가 좋으면 찾아가겠어요    3
나의 해방일지           2
주군의 태양            2
                 ..
맨투맨               1
멀리서 보면 푸른 봄       1
메모리스트             1
모범형사              1
안녕? 나야!           1
Name: count, Length: 167, dtype: int64